### **📝 Client Request: American Option Purchase**

A client has requested that the bank **purchase an American put option** on **Commonwealth Bank of Australia (CBA)** with the following details:

- **Option Type**: American Put  
- **Underlying Asset**: Commonwealth Bank of Australia (CBA)  
- **Strike Price**: $170.00  
- **Expiry Date**: 15 May 2026

To solve this problem, binomial tree method was applied to value the American put option on CBA shares. This model simulates potential future price movements of the stock over discrete time intervals until the option’s expiry on 15 May 2026. Using the Cox-Ross-Rubinstein framework, the model incorporates early exercise features and the time value of money to determine the fair premium the bank should pay for the option with a strike price of $170.00.

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import yfinance as yf
import pandas as pd
import datetime

In [ ]:
def get_CBA_data():
    
    ticker = yf.ticker('CBA.AX')
    hist = ticker.history(start="2025-05-16", end="2025-05-17")

    if hist.empty:
        raise ValueError("No price data available for 16 May 2025.")
    
    S0 = hist['Close'].iloc[0]  # closing price on 16 May 2025

    try:
        info = ticker.info
        dividend_yield = info.get("dividendYield", 0.0) or 0.0  # annual dividend yield
    except:
        dividend_yield = 0.0

    return S0, dividend_yield # returns the closing price and dividend yield